# Wiki Scrape notebook
mpkt

In [ ]:
### Disable autoscroll

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from bs4.element import Comment
import urllib.request
import psycopg2
#from config import Config (not sure if I need this, maybe later, couldn't figure out how to import)
import sys
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd


In [2]:
res = requests.get("https://en.wikipedia.org/wiki/Category:Women_editors")
soupW = bs(res.text, "html.parser")
Women_Editors = {}
Women_URLs = []
bHaveFirst = False
bHasLast = False
# /wiki/Josephine_Cables_Aldrich (so I know what the first name should be)
# /wiki/Vera_Zouroff (so I know what the last name should be)
for link in soupW.find_all("a"):
    url = link.get("href", "")
    if not bHaveFirst and "/wiki/Josephine_Cables_Aldrich" in url:
        bHaveFirst = True
    if not bHasLast and "/wiki/Vera_Zouroff" in url:
        bHasLast= True
    if bHaveFirst and "/wiki/" in url:
        Women_Editors[link.text.strip()] = url
        Women_URLs.append(url)
        #print(url)
    if bHasLast:
        break
        

In [3]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [4]:
def text_from_html(body):
    soup = bs(body, 'html.parser')
    texts = soup.findAll(text = True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)


In [5]:
def read_Women_pages(Women_Editors):
    EditorRows = []
    pref = "https://en.wikipedia.org"  # MPKT (*fixed bug here)
    i = 1
    for OneURL in Women_Editors:
        print('\n\n\n #######') # MPKT
        queryFrom = pref+OneURL # MPKT
        print(queryFrom) # MPKT
        html = urllib.request.urlopen(pref+OneURL).read()
        longText = text_from_html(html)
        #print(longText)
        EditorRows.append((i, OneURL, longText))
        print('************************************') # MPKT
        print(longText) # MPKT
        i = i+1
        if i > 5:
            break
    return EditorRows


In [6]:
def Insert_Author(cur, TableName, OneTup):
    #strCommand = "INSERT INTO " + TableName +  "VALUES(" + str(OneTup[0]) + ", \"" + OneTup[1] + "\", \"" + OneTup[2] + "\")" 
    
    #strName = "\"" + OneTup[1] + "\""
    #strYuk = "\"" + OneTup[2] + "\""
    #strCommand = "INSERT INTO " + TableName + " VALUES(" + str(OneTup[0]) + ","+ strName + "," + strYuk + ")"
    
    #strCommand = "INSERT INTO " + TableName +  "VALUES(str(OneTup[0]), ""\"" + OneTup[1] + "\", \"" + OneTup[2] + "\")" 
    #print(strCommand)
    #cur.execute(strCommand)
    
    query = "INSERT INTO " + TableName + "(Id, Name, BigText) VALUES (%s, %s, %s);"
    #query = "INSERT INTO " + TableName + "(Name, BigText) VALUES (%s, %s);"
    cur.execute(query, OneTup) 
    
    print(query)



In [7]:
def Build_DB(Editors):

    con = None

    try:        
        con = psycopg2.connect("dbname='profiles' user='marieskoczylas'")   

        cur = con.cursor()

        # text field may not be large enough. Look at it later
        cur.execute("CREATE TABLE WomenEditors3(Id INTEGER, Name VARCHAR(120), BigText VARCHAR(32000))")
        # WomenEditors
        
        for OneEd in Editors:
            Insert_Author(cur, "WomenEditors3", OneEd)
        
        #cur.execute("INSERT INTO Cars VALUES(1,'Audi',52642)")
        
        con.commit()


    #except psycopg2.DatabaseError, e:
    #    if con:
    #        con.rollback()
    #    print 'Error %s' % e    
    #    sys.exit(1)


    finally:

        if con:
            con.close()
    
    

## Scrape the pages

In [8]:
Editors = read_Women_pages(Women_URLs)
Build_DB(Editors)




 #######
https://en.wikipedia.org/wiki/Josephine_Cables_Aldrich
************************************
         Josephine Cables Aldrich   From Wikipedia, the free encyclopedia    Jump to navigation  Jump to search     Josephine Cables Aldrich    Josephine Cables Aldrich (June 12, 1843 – August 12, 1917) was an American spiritualist, Theosophist , editor, and publisher from theh U.S. state of Connecticut . Married to the politician, William F. Aldrich , she was the editor of The Occult World and the publisher of Matilda Joslyn Gage 's The Liberal Thinker . [1]     Contents    1  Early years and education  2  Career  3  Personal life  4  References   4.1  Attribution  4.2  Bibliography    5  External links     Early years and education [ edit ]  Josephine Cables was born in Litchfield, Connecticut . Her mother died when she was young, leaving her in the care of two Puritan grandmothers. They believed that a free use of the rod was necessary to save the child's soul from destruction. Th

************************************
         Johari Amini   From Wikipedia, the free encyclopedia    Jump to navigation  Jump to search        This article needs additional citations for verification . Please help improve this article by adding citations to reliable sources . Unsourced material may be challenged and removed.  (February 2016)  ( Learn how and when to remove this template message )     Overview [ edit ]  Johari Amini was an African American woman who cofounded the Third World Press in 1967. In addition to this she also contributed to other Black Arts Movement institutions such as the Writers Workshop of the Organization of Black American Culture (OBAC), its publication NOMMO, the Kuumba Theater, and the Gwendolyn Brooks Writers’ Workshop . She also worked for Black Books Bulletin as a writer as well as an editor. She has written many poems and short stories published in journals such as Black World. In addition to this impressive career in literature, she was also a pra

************************************
         Janelle Asselin   From Wikipedia, the free encyclopedia    Jump to navigation  Jump to search  Janelle Asselin is an American comic book editor and writer, known for creating the now defunct publishing company Rosy Press. Asselin currently works as a claims adjuster and currently resides in Omaha, Nebraska . [1]     Contents    1  Personal life  2  Early career   2.1  Hire This Woman  2.2  Teen Titan #1 Cover Review  2.3  Rosy Press and Fresh Romance    3  References  4  External links     Personal life [ edit ]  Asselin was born on April 10, 1983 and grew up in Malvern, Iowa , where she attended Malvern Community Junior Senior High School. She later attended Southern New Hampshire University , where she graduated in 2009 with a bachelor's degree in English, and went on to receive a master's degree in Publishing in 2011 from Pace University .  Asselin has suffered from health issues such as depression and fibromyalgia . [2] [3]  Early caree

OperationalError: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/tmp/.s.PGSQL.5432"?


## Store in database

In [ ]:
Editors = read_Women_pages(Women_URLs)
Build_DB(Editors)

dbname = 'profiles'
username = 'postgres'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

con = psycopg2.connect("dbname='profiles' user='postgres'")   
cur = con.cursor()

In [ ]:

dbname = 'profiles'
username = 'postgres'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

con = psycopg2.connect("dbname='profiles' user='postgres'")   
cur = con.cursor()

# query:
sql_query = """
SELECT * FROM WomenEditors3;
"""
view_from_sql = pd.read_sql_query(sql_query,con)
view_from_sql.head()

# Why can't I see what's in the "bigtext" section...?
sql_query = """
SELECT 'bigtext' FROM WomenEditors3 WHERE id='2';
"""
view_from_sql = pd.read_sql_query(sql_query,con)
view_from_sql.head()
